In [1]:
# compute taxanomy abundance difference
import sys
sys.path.append('..')
import abd_profile
import os
import pandas as pd
import copy
import numpy as np
from statsmodels.stats.multitest import fdrcorrection as fdr
from scipy.stats import mannwhitneyu

In [2]:
def compare(pheno_profiles, p_line):    
    pheno_list = list(pheno_profiles.keys())
    disease = ''
    for pheno in pheno_list:
        if pheno != 'Health':
            disease = pheno
    p_df = pd.DataFrame(columns=['mean(case)', 'mean(control)', 'var(case)', 'var(control)', 'fc(case/control)', 'diff(case-control)', 'abs', 'non_zero(control)', 'non_zero(case)', 'non_zero%(control)', 'non_zero%(case)', 'p_adj'], index=list(pheno_profiles[disease].columns))

    # check each taxon
    for sp in pheno_profiles[disease].columns:
        disease_v = list(pheno_profiles[disease][sp])
        control_v = list(pheno_profiles['Health'][sp])
        p_df.loc[sp, 'mean(case)'] = np.mean(disease_v)
        p_df.loc[sp, 'mean(control)'] = np.mean(control_v)
        p_df.loc[sp, 'var(case)'] = np.var(disease_v)
        p_df.loc[sp, 'var(control)'] = np.var(control_v)
        p_df.loc[sp, 'diff(case-control)'] = p_df.loc[sp, 'mean(case)'] - p_df.loc[sp, 'mean(control)']
        p_df.loc[sp, 'fc(case/control)'] = p_df.loc[sp, 'mean(case)']/p_df.loc[sp, 'mean(control)']
        p_df.loc[sp, 'non_zero(case)'] = len(disease_v) - disease_v.count(0)
        p_df.loc[sp, 'non_zero(control)'] = len(control_v) - control_v.count(0)
        p_df.loc[sp, 'non_zero%(case)'] = p_df.loc[sp, 'non_zero(case)']/len(disease_v)
        p_df.loc[sp, 'non_zero%(control)'] = p_df.loc[sp, 'non_zero(control)']/len(control_v)
    p_df['abs'] = abs(p_df['diff(case-control)'])

    for cluster in p_line.index:
        p_df.loc[cluster, 'p_adj'] = p_line[cluster]


    return p_df


In [3]:
p_cutoff = 0.05
plist = ['ACVD',
 'BD',
 'CRC',
 'IBD',
 'IGT',
 'CFS',
 'STH',
 'T2D',
 'adenoma',
 'asthma',
 'carcinoma_surgery_history',
 'hypertension',
 'migraine',
 'schizofrenia']

outer = '../result/GCN_fix_tree/SE/GCN_tree_diff'
all_cohort_df = pd.read_csv(os.path.join(outer, 'p_all_cohorts.tsv'), sep='\t', index_col=0)
for project in plist:
    out = os.path.join(outer, project)
    projects_dir = os.path.join(outer, project)
    for dir in os.listdir(projects_dir):
        indir = os.path.join(projects_dir, dir)
        pheno_profiles = {}
        for pheno in os.listdir(indir):
            if not pheno.startswith('se'):
                continue
            pheno_short = pheno.split('.')[0].split('_')[1]
            pheno_profiles[pheno_short] = pd.read_csv(os.path.join(indir, pheno), sep='\t', index_col=0)

        if len(list(pheno_profiles.keys())) >1:
            p_df = compare(pheno_profiles, all_cohort_df.loc[dir, ])
            outpath = os.path.join(indir, 'p_{}.tsv'.format(dir))
            p_df.to_csv(outpath, sep='\t', index=True)
            

<ipython-input-2-6f284588b888>:18: RuntimeWarning: divide by zero encountered in scalar divide
  p_df.loc[sp, 'fc(case/control)'] = p_df.loc[sp, 'mean(case)']/p_df.loc[sp, 'mean(control)']
<ipython-input-2-6f284588b888>:18: RuntimeWarning: invalid value encountered in scalar divide
  p_df.loc[sp, 'fc(case/control)'] = p_df.loc[sp, 'mean(case)']/p_df.loc[sp, 'mean(control)']
<ipython-input-2-6f284588b888>:18: RuntimeWarning: invalid value encountered in scalar divide
  p_df.loc[sp, 'fc(case/control)'] = p_df.loc[sp, 'mean(case)']/p_df.loc[sp, 'mean(control)']
<ipython-input-2-6f284588b888>:18: RuntimeWarning: divide by zero encountered in scalar divide
  p_df.loc[sp, 'fc(case/control)'] = p_df.loc[sp, 'mean(case)']/p_df.loc[sp, 'mean(control)']
<ipython-input-2-6f284588b888>:18: RuntimeWarning: invalid value encountered in scalar divide
  p_df.loc[sp, 'fc(case/control)'] = p_df.loc[sp, 'mean(case)']/p_df.loc[sp, 'mean(control)']
<ipython-input-2-6f284588b888>:18: RuntimeWarning: divide 